# Evaluate

In [1]:
import argparse
import torch
import glob
from sklearn.metrics import average_precision_score, accuracy_score, roc_auc_score, precision_recall_curve

from dassl.utils import setup_logger, set_random_seed, collect_env_info
from dassl.config import get_cfg_default
from dassl.engine import build_trainer

# custom

import datasets.imagenet
import datasets.guided
import datasets.biggan
import datasets.cyclegan
import datasets.dalle2
import datasets.deepfake
import datasets.gaugan
import datasets.glide_50_27
import datasets.glide_100_10
import datasets.glide_100_27
import datasets.ldm_100
import datasets.ldm_200
import datasets.ldm_200_cfg
import datasets.stargan
import datasets.stylegan
import datasets.stylegan2
import datasets.stylegan3
import datasets.sd_512x512
import datasets.sdxl
import datasets.dalle3
import datasets.taming
import datasets.eg3d
import datasets.firefly
import datasets.midjourney_v5
import datasets.progan
import datasets.faceswap


import trainers.coop
import trainers.clip_adapter
import trainers.clip_zero_shot
import trainers.cocoop
import trainers.zsclip

In [2]:
# from eval_utils import print_args, reset_cfg, extend_cfg, setup_cfg, get_parsed_args

from eval_utils_fine_tuned import print_args, reset_cfg, extend_cfg, setup_cfg, get_parsed_args
# from eval_utils_zero_shot import print_args, reset_cfg, extend_cfg, setup_cfg, get_parsed_args

In [3]:
# model_dirs = sorted(glob.glob('../CoOp_Trained/selected_models/*/'))
# model_dirs = [path.replace('\\','/') for path in model_dirs]
# model_dirs

model_dirs = sorted(glob.glob('../finetuned_clip/finetuned_clips/*/'))
model_dirs = [path.replace('\\','/') for path in model_dirs]
model_dirs = model_dirs[::-1]
model_dirs

['../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/',
 '../finetuned_clip/finetuned_clips/finetuned_2_epoch_80k/',
 '../finetuned_clip/finetuned_clips/finetuned_2_epoch_60k/',
 '../finetuned_clip/finetuned_clips/finetuned_2_epoch_40k/',
 '../finetuned_clip/finetuned_clips/finetuned_2_epoch_20k/']

In [4]:
# model_dirs[1].split('/')[-2].split('_')[0]

In [5]:
dataset_names = ['progan', 'biggan', 'cyclegan', 'eg3d', 'gaugan',  'stargan', 'stylegan', 'stylegan2', 'stylegan3', 
                 'dalle2', 'glide_50_27', 'glide_100_10', 'glide_100_27', 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
                 'sd_512x512', 'sdxl', 'taming', 'deepfake', 'firefly', 'midjourney_v5', 'dalle3', 'faceswap']
# dataset_names = ['deepfake']
                #  'eg3d', 'stylegan3'] 'dalle2', 'deepfake', 'glide_50_27', 'glide_100_10', 'glide_100_27',
                # 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
# dataset_names = ['firefly', 'midjourney_v5', 'dalle3']

In [6]:
import sys
sys.argv = ['']

In [7]:
import json

def update_and_save_evaluation(model_name, dataset_name, accuracy, f1_score, average_precision):
    global model_evaluations
    
    # Check if the model key exists in the dictionary
    if model_name not in model_evaluations:
        model_evaluations[model_name] = {}
    
    # Check if the dataset key exists for the given model
    if dataset_name not in model_evaluations[model_name]:
        model_evaluations[model_name][dataset_name] = {}
    
    # Update evaluation results
    model_evaluations[model_name][dataset_name]["accuracy"] = accuracy
    model_evaluations[model_name][dataset_name]["f1_score"] = f1_score
    model_evaluations[model_name][dataset_name]["average_precision"] = average_precision
    
    # Save the updated dictionary to a JSON file
    with open("finetuned_model_evaluations_2_sigma.json", "w") as json_file:
        json.dump(model_evaluations, json_file, indent=2)

In [8]:
# # model_evaluations = {}
# # for model in model_dirs:
# #     num_ctx_tokens = int(model.split('/')[-2].split('_')[1])
# #     print(num_ctx_tokens)
# #     for dataset in dataset_names:
# #         args = get_parsed_args(model, dataset, num_ctx_tokens)
# #         cfg = setup_cfg(args)
# #         print("Setting fixed seed: {}".format(cfg.SEED))
# #         set_random_seed(cfg.SEED)
# #         if torch.cuda.is_available() and cfg.USE_CUDA:
# #             print('Using CUDA!!!')
# #             torch.backends.cudnn.benchmark = True

# #         print_args(args, cfg)
# #         print("Collecting env info ...")
# #         print("** System info **\n{}\n".format(collect_env_info()))

# #         trainer = build_trainer(cfg)
# #         trainer.load_model(args.model_dir, epoch=args.load_epoch)

# #         results, results_dict = trainer.test()
# #         update_and_save_evaluation(model, dataset, results_dict['accuracy'], results_dict['macro_f1'], results_dict['average_precision'])


model_evaluations = {}
for model in model_dirs:
    print(model)
    num_ctx_tokens = 0
    print(num_ctx_tokens)
    for dataset in dataset_names:
        args = get_parsed_args(model, dataset)
        cfg = setup_cfg(args)
        print("Setting fixed seed: {}".format(cfg.SEED))
        set_random_seed(cfg.SEED)
        if torch.cuda.is_available() and cfg.USE_CUDA:
            print('Using CUDA!!!')
            torch.backends.cudnn.benchmark = True

        print_args(args, cfg)
        print("Collecting env info ...")
        print("** System info **\n{}\n".format(collect_env_info()))

        trainer = build_trainer(cfg)
        trainer.load_model(args.model_dir, epoch=args.load_epoch)
        print(trainer.model.dtype)
        results, results_dict = trainer.test()
        update_and_save_evaluation(model, dataset, results_dict['accuracy'], results_dict['macro_f1'], results_dict['average_precision'])
    break

../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/
0
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/progan.yaml
eval_only: True
head: 
load_epoch: 1
model_dir: ../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/
no_train: False
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CLIP_Adapter
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: Fals

100%|██████████| 80/80 [02:08<00:00,  1.60s/it]


=> result
* total: 8,000
* correct: 5,607
* average_precision: 95.4%
* accuracy: 70.1%
* error: 29.9%
* macro_f1: 67.2%
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/biggan.yaml
eval_only: True
head: 
load_epoch: 1
model_dir: ../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/
no_train: False
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CLIP_Adapter
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER:

100%|██████████| 40/40 [01:16<00:00,  1.91s/it]


=> result
* total: 4,000
* correct: 2,305
* average_precision: 81.8%
* accuracy: 57.6%
* error: 42.4%
* macro_f1: 48.4%
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/cyclegan.yaml
eval_only: True
head: 
load_epoch: 1
model_dir: ../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/
no_train: False
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CLIP_Adapter
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLE

...

=> result
* total: 2,000
* correct: 1,251
* average_precision: 83.8%
* accuracy: 62.5%
* error: 37.5%
* macro_f1: 57.0%
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/dalle3.yaml
eval_only: True
head: 
load_epoch: 1
model_dir: ../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/
no_train: False
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CLIP_Adapter
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER:

100%|██████████| 20/20 [00:51<00:00,  2.57s/it]


=> result
* total: 2,000
* correct: 1,184
* average_precision: 78.6%
* accuracy: 59.2%
* error: 40.8%
* macro_f1: 54.4%
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/faceswap.yaml
eval_only: True
head: 
load_epoch: 1
model_dir: ../finetuned_clip/finetuned_clips/finetuned_from_own_pc_1_epoch_100k/
no_train: False
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CLIP_Adapter
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLE

100%|██████████| 56/56 [01:37<00:00,  1.73s/it]

=> result
* total: 5,600
* correct: 3,208
* average_precision: 58.8%
* accuracy: 57.3%
* error: 42.7%
* macro_f1: 57.3%


In [9]:
import re
s = model_dirs[0].split('/')[-2].split('_')[1]
int(re.split('(\d+)',s)[1])

16

In [11]:
model_evaluations = {}
for model in model_dirs:
    splitted_string = model.split('/')[-2].split('_')[1]
    num_ctx_tokens = int(re.split('(\d+)',splitted_string)[1])
    print(num_ctx_tokens)
    for dataset in dataset_names:
        args = get_parsed_args(model, dataset, num_ctx_tokens)
        cfg = setup_cfg(args)
        print("Setting fixed seed: {}".format(cfg.SEED))
        set_random_seed(cfg.SEED)
        if torch.cuda.is_available() and cfg.USE_CUDA:
            print('Using CUDA!!!')
            torch.backends.cudnn.benchmark = True

        print_args(args, cfg)
        print("Collecting env info ...")
        print("** System info **\n{}\n".format(collect_env_info()))

        trainer = build_trainer(cfg)
        trainer.load_model(args.model_dir, epoch=args.load_epoch)

        results, results_dict = trainer.test()
        update_and_save_evaluation(model, dataset, results_dict['accuracy'], results_dict['macro_f1'], results_dict['average_precision'])


16
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/faceswap.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: ../CoOp_Trained/selected_models/100000_16context_best_until_now/
no_train: False
num_ctx_tokens: 16
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CoOp
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_TYPE: 
  NAME: faceswap
  NUM_

100%|██████████| 56/56 [00:53<00:00,  1.06it/s]


=> result
* total: 5,600
* correct: 4,181
* average_precision: 88.0%
* accuracy: 74.7%
* error: 25.3%
* macro_f1: 73.6%
4
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/faceswap.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: ../CoOp_Trained/selected_models/100000_4context/
no_train: False
num_ctx_tokens: 4
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CoOp
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: Random

100%|██████████| 56/56 [00:52<00:00,  1.07it/s]


=> result
* total: 5,600
* correct: 3,878
* average_precision: 83.6%
* accuracy: 69.2%
* error: 30.8%
* macro_f1: 66.9%
8
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/faceswap.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: ../CoOp_Trained/selected_models/100000_8context/
no_train: False
num_ctx_tokens: 8
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CoOp
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: Random

...

=> result
* total: 5,600
* correct: 3,852
* average_precision: 84.6%
* accuracy: 68.8%
* error: 31.2%
* macro_f1: 65.8%
4
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/faceswap.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: ../CoOp_Trained/selected_models/30000_4context/
no_train: False
num_ctx_tokens: 4
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CoOp
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomS

100%|██████████| 56/56 [00:53<00:00,  1.04it/s]


=> result
* total: 5,600
* correct: 3,371
* average_precision: 86.1%
* accuracy: 60.2%
* error: 39.8%
* macro_f1: 52.8%
8
Setting fixed seed: 17
Using CUDA!!!
***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/faceswap.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: ../CoOp_Trained/selected_models/30000_8context/
no_train: False
num_ctx_tokens: 8
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CoOp
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 16
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomS

 27%|██▋       | 15/56 [00:33<01:31,  2.22s/it]


KeyboardInterrupt: 

In [ ]:
print('==============UNTIL HERE=================')

In [6]:
args = get_parsed_args(model_dirs[0], 'guided')
cfg = setup_cfg(args)
print("Setting fixed seed: {}".format(cfg.SEED))
set_random_seed(cfg.SEED)
if torch.cuda.is_available() and cfg.USE_CUDA:
    print('Using CUDA!!!')
    torch.backends.cudnn.benchmark = True

Setting fixed seed: 17
Using CUDA!!!


In [7]:
print_args(args, cfg)
print("Collecting env info ...")
print("** System info **\n{}\n".format(collect_env_info()))

***************
** Arguments **
***************
backbone: 
config_file: ../CoOp/configs/trainers/coop/vit_l14_ep2.yaml
dataset_config_file: ../CoOp/configs/datasets/guided.yaml
eval_only: True
head: 
load_epoch: 2
model_dir: ../CoOp_Trained/selected_models/100000_16context_best_until_now/
no_train: False
opts: []
output_dir: ../CoOp/outputs/
resume: 
root: ../Datasets/ICMRDataset/test/deepfake_eval/
seed: 17
source_domains: None
target_domains: None
trainer: CoOp
transforms: None
************
** Config **
************
DATALOADER:
  K_TRANSFORMS: 1
  NUM_WORKERS: 8
  RETURN_IMG0: False
  TEST:
    BATCH_SIZE: 100
    SAMPLER: SequentialSampler
  TRAIN_U:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAME_AS_X: True
    SAMPLER: RandomSampler
  TRAIN_X:
    BATCH_SIZE: 32
    N_DOMAIN: 0
    N_INS: 16
    SAMPLER: RandomSampler
DATASET:
  ALL_AS_UNLABELED: False
  CIFAR_C_LEVEL: 1
  CIFAR_C_TYPE: 
  NAME: guided
  NUM_LABELED: -1
  NUM_SHOTS: -1
  ROOT: ../Datasets/ICMRDataset/tes

In [8]:
trainer = build_trainer(cfg)
trainer.load_model(args.model_dir, epoch=args.load_epoch)

Loading trainer: CoOp
Loading dataset: guided
Building transform_train
+ random resized crop (size=(224, 224), scale=(0.08, 1.0))
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
Building transform_test
+ resize the smaller edge to 224
+ 224x224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
---------  ------
Dataset    guided
# classes  2
# train_x  2,000
# val      2,000
# test     2,000
---------  ------
Loading CLIP (backbone: ViT-L/14)
Building custom CLIP
Initializing a generic context
Initial context: "X X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
Turning off gradients in both the image and the text encoder
Loading evaluator: Classification
Loading weights to prompt_learner from "../CoOp_Trained/selected_models/100000_16context_best_until_now/prompt_learner\m

In [9]:
results, results_dict = trainer.test()

Evaluate on the *test* set


  5%|▌         | 1/20 [00:27<08:40, 27.38s/it]

[0.00439453125, 0.0009765625, 0.00146484375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00146484375, 0.00048828125, 0.00439453125, 0.0048828125, 0.00048828125, 0.009765625, 0.00244140625, 0.0, 0.0, 0.0029296875, 0.00048828125, 0.00048828125, 0.455078125, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.0, 0.00732421875, 0.0, 0.0, 0.0, 0.0146484375, 0.00146484375, 0.00146484375, 0.0, 0.0, 0.0009765625, 0.0078125, 0.0, 0.0107421875, 0.0009765625, 0.60400390625, 0.0, 0.00390625, 0.013671875, 0.0, 0.0, 0.0, 0.310546875, 0.017578125, 0.0009765625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09716796875, 0.00244140625, 0.0, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.00146484375, 0.03955078125, 0.0, 0.0, 0.01025390625, 0.0, 0.0068359375, 0.00341796875, 0.06201171875, 0.0, 0.01513671875, 0.0, 0.01171875, 0.30419921875, 0.00244140625, 0.0, 0.0, 0.00146484375, 0.0, 0.0, 0.0, 0.001953125, 0.0, 0.0009765625, 0.0, 0.00048828125, 0.0, 0.0146484375, 0.00048828125, 0.0, 0.0, 0.00048828125, 0.00048828125]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

 10%|█         | 2/20 [00:27<03:27, 11.55s/it]

[0.00439453125, 0.0009765625, 0.00146484375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00146484375, 0.00048828125, 0.00439453125, 0.0048828125, 0.00048828125, 0.009765625, 0.00244140625, 0.0, 0.0, 0.0029296875, 0.00048828125, 0.00048828125, 0.455078125, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.0, 0.00732421875, 0.0, 0.0, 0.0, 0.0146484375, 0.00146484375, 0.00146484375, 0.0, 0.0, 0.0009765625, 0.0078125, 0.0, 0.0107421875, 0.0009765625, 0.60400390625, 0.0, 0.00390625, 0.013671875, 0.0, 0.0, 0.0, 0.310546875, 0.017578125, 0.0009765625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09716796875, 0.00244140625, 0.0, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.00146484375, 0.03955078125, 0.0, 0.0, 0.01025390625, 0.0, 0.0068359375, 0.00341796875, 0.06201171875, 0.0, 0.01513671875, 0.0, 0.01171875, 0.30419921875, 0.00244140625, 0.0, 0.0, 0.00146484375, 0.0, 0.0, 0.0, 0.001953125, 0.0, 0.0009765625, 0.0, 0.00048828125, 0.0, 0.0146484375, 0.00048828125, 0.0, 0.0, 0.00048828125, 0.00048828125, 0.0029296875, 0.0048828125, 0.00

...

[0.00439453125, 0.0009765625, 0.00146484375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00146484375, 0.00048828125, 0.00439453125, 0.0048828125, 0.00048828125, 0.009765625, 0.00244140625, 0.0, 0.0, 0.0029296875, 0.00048828125, 0.00048828125, 0.455078125, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.0, 0.00732421875, 0.0, 0.0, 0.0, 0.0146484375, 0.00146484375, 0.00146484375, 0.0, 0.0, 0.0009765625, 0.0078125, 0.0, 0.0107421875, 0.0009765625, 0.60400390625, 0.0, 0.00390625, 0.013671875, 0.0, 0.0, 0.0, 0.310546875, 0.017578125, 0.0009765625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09716796875, 0.00244140625, 0.0, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.00146484375, 0.03955078125, 0.0, 0.0, 0.01025390625, 0.0, 0.0068359375, 0.00341796875, 0.06201171875, 0.0, 0.01513671875, 0.0, 0.01171875, 0.30419921875, 0.00244140625, 0.0, 0.0, 0.00146484375, 0.0, 0.0, 0.0, 0.001953125, 0.0, 0.0009765625, 0.0, 0.00048828125, 0.0, 0.0146484375, 0.00048828125, 0.0, 0.0, 0.00048828125, 0.00048828125, 0.0029296875, 0.0048828125, 0.00

100%|██████████| 20/20 [00:36<00:00,  2.01it/s]

[0.00439453125, 0.0009765625, 0.00146484375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00146484375, 0.00048828125, 0.00439453125, 0.0048828125, 0.00048828125, 0.009765625, 0.00244140625, 0.0, 0.0, 0.0029296875, 0.00048828125, 0.00048828125, 0.455078125, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.0, 0.00732421875, 0.0, 0.0, 0.0, 0.0146484375, 0.00146484375, 0.00146484375, 0.0, 0.0, 0.0009765625, 0.0078125, 0.0, 0.0107421875, 0.0009765625, 0.60400390625, 0.0, 0.00390625, 0.013671875, 0.0, 0.0, 0.0, 0.310546875, 0.017578125, 0.0009765625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09716796875, 0.00244140625, 0.0, 0.0, 0.00048828125, 0.0, 0.0, 0.0, 0.00146484375, 0.03955078125, 0.0, 0.0, 0.01025390625, 0.0, 0.0068359375, 0.00341796875, 0.06201171875, 0.0, 0.01513671875, 0.0, 0.01171875, 0.30419921875, 0.00244140625, 0.0, 0.0, 0.00146484375, 0.0, 0.0, 0.0, 0.001953125, 0.0, 0.0009765625, 0.0, 0.00048828125, 0.0, 0.0146484375, 0.00048828125, 0.0, 0.0, 0.00048828125, 0.00048828125, 0.0029296875, 0.0048828125, 0.00

100%|██████████| 20/20 [00:37<00:00,  1.86s/it]

=> result
* total: 2,000
* correct: 1,686
* average_precision: 97.4%
* accuracy: 84.3%
* error: 15.7%
* macro_f1: 83.9%


In [37]:
(results_dict)

OrderedDict([('accuracy', 78.44699777613047),
             ('error_rate', 21.553002223869527),
             ('macro_f1', 77.47484357151382),
             ('average_precision', 92.59029414434582)])

In [42]:
model_name = args.model_dir.split('/')[-2]

In [43]:
# Initialize an empty dictionary to store model evaluations
model_evaluations = {}

In [44]:
import json

def update_and_save_evaluation(model_name, dataset_name, accuracy, f1_score, average_precision):
    global model_evaluations
    
    # Check if the model key exists in the dictionary
    if model_name not in model_evaluations:
        model_evaluations[model_name] = {}
    
    # Check if the dataset key exists for the given model
    if dataset_name not in model_evaluations[model_name]:
        model_evaluations[model_name][dataset_name] = {}
    
    # Update evaluation results
    model_evaluations[model_name][dataset_name]["accuracy"] = accuracy
    model_evaluations[model_name][dataset_name]["f1_score"] = f1_score
    model_evaluations[model_name][dataset_name]["average_precision"] = average_precision
    
    # Save the updated dictionary to a JSON file
    with open("model_evaluations.json", "w") as json_file:
        json.dump(model_evaluations, json_file, indent=2)

# # Example usage after evaluating a model
# update_and_save_evaluation("model1", "dataset1", 0.85, 0.78, 0.92)
# update_and_save_evaluation("model1", "dataset2", 0.92, 0.89, 0.95)

# update_and_save_evaluation("model2", "dataset1", 0.78, 0.67, 0.88)
# update_and_save_evaluation("model2", "dataset2", 0.91, 0.87, 0.93)


In [45]:
update_and_save_evaluation(cfg['TRAINER']['NAME']+str('_')+model_name, 'gaugan', results_dict['accuracy'], results_dict['macro_f1'], results_dict['average_precision'])

In [51]:
model_evaluations['CoOp_100000_16context_best_until_now']['gaugan']

{'accuracy': 99.65,
 'f1_score': 99.64999571244748,
 'average_precision': 99.99919799075577}

In [37]:
cfg['TRAINER']['NAME']

'CoOp'

In [17]:
outs = trainer.model(torch.rand(1, 3, 224, 224).cuda())

In [28]:
outs

tensor([[13.9297, 14.6406]], device='cuda:0', dtype=torch.float16,
       grad_fn=<MmBackward0>)

In [11]:
from torchvision.transforms import (
    Resize, Compose, ToTensor, Normalize, CenterCrop, RandomCrop, ColorJitter,
    RandomApply, GaussianBlur, RandomGrayscale, RandomResizedCrop,
    RandomHorizontalFlip
)
from torchvision.transforms.functional import InterpolationMode
INTERPOLATION_MODES = {
    "bilinear": InterpolationMode.BILINEAR,
    "bicubic": InterpolationMode.BICUBIC,
    "nearest": InterpolationMode.NEAREST,
}

In [12]:
def _build_transform_test(cfg, choices):
    print("Building transform_test")
    tfm_test = []
    target_size = 224
    interp_mode = INTERPOLATION_MODES[cfg.INPUT.INTERPOLATION]
    input_size = cfg.INPUT.SIZE

    print(f"+ resize the smaller edge to {max(input_size)}")
    tfm_test += [Resize(max(input_size), interpolation=interp_mode)]

    print(f"+ {target_size} center crop")
    tfm_test += [CenterCrop(input_size)]

    print("+ to torch tensor of range [0, 1]")
    tfm_test += [ToTensor()]
    
    normalize = Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])

    if "normalize" in choices:
        print(
            f"+ normalization (mean={cfg.INPUT.PIXEL_MEAN}, std={cfg.INPUT.PIXEL_STD})"
        )
        tfm_test += [normalize]

    tfm_test = Compose(tfm_test)

    return tfm_test

In [13]:
# Building transform_test
# + resize the smaller edge to 224
# + 224x224 center crop
# + to torch tensor of range [0, 1]
# + normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])

In [14]:
tfms = _build_transform_test(cfg, choices=['normalize'])

Building transform_test
+ resize the smaller edge to 224
+ 224 center crop
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])


In [15]:
trainer.model.eval()

CustomCLIP(
  (prompt_learner): PromptLearner()
  (image_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear

In [26]:
import cv2
from PIL import Image
from torchvision import datasets, transforms
import numpy as np

device = ['cuda']
labels_map = ["real", "fake"]
count = 0
all_images = []
predicted_labels = []
true_labels = []
predicted_probs = []

images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/deepfake/0_real/*.png'))
fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/deepfake/1_fake/*.png'))

# images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01440764/*.jpg'))
# fake_images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01443537/*.png'))

print(len(images))
print(len(fake_images))
# 
# np.random.shuffle(fake_images)
# np.random.shuffle(fake_images)

all_images = images
all_images.extend(fake_images)

all_images = [path.replace('\\','/') for path in all_images]
y_pred = []
for image in all_images:
    img = cv2.imread(image)
    # img = cv2_jpg(img, 50)
    # image = add_noise(image, 0.4)
    img = Image.fromarray(img)
    img = img.convert('RGB')
    # img = Image.open(image)
    # tfms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),
    #                            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    #                           ])
    img = tfms(img)
    img = img.cuda()
    
    with torch.no_grad():
        outputs = trainer.model(img.unsqueeze(0))
        print(outputs)
    for idx in torch.topk(outputs[0], k=1).indices.tolist():
        prob = torch.softmax(outputs[0], 0)[idx].item()
        if labels_map[idx] == 'real':
            print("real")
            predicted_labels.append(0)
            predicted_probs.append(1 - prob)
        else:
            print("fake")
            predicted_labels.append(1)
            predicted_probs.append(prob)
    
    # print(outputs.sigmoid().flatten())
    # if outputs.sigmoid().flatten() < 0.5:
    #     print("real")
    #     predicted_labels.append(0)
    #     predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
    # else:
    #     print("fake")
    #     predicted_labels.append(1)
    #     predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
    
    print(image.split('/')[-2])
    if 'real' in image.split('/')[-2]:
        true_labels.append(0)
    else:
        true_labels.append(1)
    print('--------------')

y_true, y_pred = np.array(true_labels), np.array(y_pred)
# train with compression and blur

2698
2698
tensor([[16.1250,  9.7344]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[16.0469,  9.8828]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[16.0938,  9.3438]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[15.4219, 11.3516]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[14.5703, 10.9609]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[13.2578,  9.2188]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[11.7812,  9.0312]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[11.9688,  7.2148]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[12.0781,  8.0234]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[11.7422,  9.0469]], device='cuda:0', dtype=torch.float16)
real
0_real
--------------
tensor([[8.4844, 6.7383]], device='cuda:0', dtype=

In [27]:
len(predicted_probs)

5396

In [28]:
average_precision = 100 * average_precision_score(true_labels, predicted_probs)

print(f"average_precision: {average_precision:.1f}%\n")

average_precision: 88.6%



In [29]:
average_precision

88.56107856510411

In [30]:
accuracy_score(true_labels, predicted_labels)

0.7607487027427724

In [31]:
count = 0
for i in range(len(true_labels)):
    if true_labels[i] == predicted_labels[i]:
        pass
    else:
        count+=1
        print('incorrect')

incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect


In [32]:
count

1291

In [34]:
5396 - count

4105

In [36]:
4105/5396

0.7607487027427724

In [38]:
len(predicted_probs)

5396

In [50]:
outputs.max(1)[1]

tensor([0], device='cuda:0')